In [ ]:
# imports
import torch
import torch.nn as nn

import sklearn.metrics as metrics
import matplotlib.pyplot as plt

# consts
CRITERION = nn.CrossEntropyLoss()

In [ ]:
def plot_loss(history):
    plt.plot(history)
    plt.xlabel('Batch')
    plt.ylabel('Loss')
    plt.show()

In [ ]:
def train_loop(model, train_loader, optimizer, criterion, num_epoch, device, history = []):
    model.train()

    for epoch in range(num_epoch):

        running_loss = 0
        for i, (imgs, cats) in enumerate(train_loader, 1):
            imgs, cats = imgs.to(device), cats.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)

            loss = criterion(outputs, cats)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if not i % 10:
                print(f"Epoch {epoch+1}, Batch {i}, Loss: {running_loss/10:.4f}")
                history.append(running_loss/10)
                running_loss = 0.0
    
    print("Finished training")

In [ ]:
def eval(model, test_loader, device):
    model.eval()

    total = 0
    correct = 0

    pred = []
    real = []

    with torch.no_grad():
        for imgs, cats in test_loader:
            imgs, cats = imgs.to(device), cats.to(device)

            outputs = model(imgs)
            _, predicted = torch.max(outputs, 1)

            pred.extend(predicted.tolist())
            real.extend(cats.tolist())

            total += cats.size(0)
            correct += (predicted == cats).sum().item()

    print(f'Accuracy on test images: {100 * correct / total:.2f}%')
    print(metrics.classification_report(pred, real, target_names=dataset.classes))
    
    #return pred, real